In [0]:
import pandas as pd
import random
import string
from datetime import datetime, timedelta

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import (
    StructType, StructField, StringType, IntegerType, LongType,
    FloatType, DoubleType, BooleanType, DateType, TimestampType, ArrayType
)
# Initialize SparkSession
# spark = SparkSession.builder \
#     .appName("ExcelMetadataWorkflow") \
#     .getOrCreate()

In [0]:
!pip install openpyxl

     |████████████████████████████████| 250 kB 4.3 MB/s 
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-6f923ae3-0a6e-47b9-977a-636418ce1b63/bin/python -m pip install --upgrade pip' command.


In [0]:
# -------------------------------------
# Step 1. Read all sheets from the Excel file.
# -------------------------------------
excel_path = "/tmp/HPE_NVDA_datagen.xlsx"  # update this path

# Read every sheet into a dictionary: keys are sheet names, values are DataFrames.
sheets = pd.read_excel(excel_path, sheet_name=None)
# sheets = spark.read.  
sheet_names = list(sheets.keys())
print("Found sheets:", sheet_names)

Found sheets: ['data_sources', 'joins', 'all_joins', 'table_a', 'table_b', 'table_c', 'table_d', 'table_e']


In [0]:
# -------------------------------------
# Step 2. Process the tables overview (first sheet)
# -------------------------------------
# Assumption: The first sheet (e.g. "Tables") lists the table names and approximate row counts.
tables_overview_df = sheets[sheet_names[0]]
# Adjust these column names if your Excel file uses different names.
table_names = tables_overview_df["masked_table_id"].tolist()
approx_row_counts = tables_overview_df["num_rows_approx"].tolist()

print("Tables and approximate row counts:")
for tbl, cnt in zip(table_names, approx_row_counts):
    print(f"  {tbl}: ~{cnt} rows")


Tables and approximate row counts:
  table_a: ~100000000000 rows
  table_b: ~14000 rows
  table_c: ~2100000000 rows
  table_d: ~12000000 rows
  table_e: ~33000000 rows


In [0]:
# -------------------------------------
# Step 3. Read each table's metadata (columns, types, etc.)
# -------------------------------------
# Here we assume that the sheet name for each table is the same as the table name.
table_metadata = {}
for tbl in table_names:
    if tbl in sheets:
        meta_df = sheets[tbl]
        table_metadata[tbl] = meta_df
        print(f"Loaded metadata for table '{tbl}'.")
    else:
        print(f"Warning: No metadata sheet found for table '{tbl}'.")

Loaded metadata for table 'table_a'.
Loaded metadata for table 'table_b'.
Loaded metadata for table 'table_c'.
Loaded metadata for table 'table_d'.
Loaded metadata for table 'table_e'.


In [0]:
# -------------------------------------
# Step 4. Define a mapping from your Excel type names to Spark types.
# -------------------------------------
spark_type_mapping = {
    "StringType()": StringType(),
    "StringType": StringType(),
    "IntegerType()": IntegerType(),
    "IntegerType()": IntegerType(),
    "LongType()": LongType(),
    "FloatType()": FloatType(),
    "DoubleType()": DoubleType(),
    "BooleanType()": BooleanType(),
    "BooleanType()": BooleanType(),
    "DateType()": DateType(),
    "TimestampType()": TimestampType(),
    "ArrayType(IntegerType(), True)": ArrayType(IntegerType(), True),
    "ArrayType(StringType(), True)": ArrayType(StringType(), True)
}

def create_schema(meta_df):
    """
    Create a Spark schema (StructType) from the metadata DataFrame.
    For numerical types, if "min" and "max" are provided, they are stored in the field metadata.
    This version ensures that the type from the spreadsheet is used (if it matches).
    """
    fields = []
    # Ensure that the range columns exist in the DataFrame.
    has_range = ("min" in meta_df.columns) and ("max" in meta_df.columns)
    
    for idx, row in meta_df.iterrows():
        col_name = row["masked_column_name"]
        # Convert the Type from the spreadsheet to a lower-case string.
        type_str = str(row["spark_data_type"]).strip() if pd.notna(row["spark_data_type"]) else "string"
        spark_type = spark_type_mapping.get(type_str)
        
        if spark_type is None:
            # If the type is not recognized, warn and default to StringType.
            print(f"Warning: Unrecognized type '{row['spark_data_type']}' for column '{col_name}'. Using StringType.")
            spark_type = StringType()
        
        md = {}
        # For numerical types, if min and max values are provided, store them in metadata.
        if isinstance(spark_type, (IntegerType, LongType, FloatType, DoubleType)) and has_range:
            if pd.notna(row["min"]) and pd.notna(row["max"]):
                md["min"] = row["min"]
                md["max"] = row["max"]
        
        fields.append(StructField(col_name, spark_type, True, metadata=md))
    
    return StructType(fields)

# Create a dictionary of schemas for each table.
schemas = {}
for tbl, meta_df in table_metadata.items():
    schema = create_schema(meta_df)
    schemas[tbl] = schema
    print(f"Schema for table '{tbl}': {schema}")

Schema for table 'table_a': StructType([StructField('col_a_0', StringType(), True), StructField('col_a_1', LongType(), True), StructField('col_a_2', LongType(), True), StructField('col_a_3', IntegerType(), True), StructField('col_a_4', ArrayType(IntegerType(), True), True), StructField('col_a_5', LongType(), True), StructField('col_a_6', LongType(), True), StructField('col_a_7', IntegerType(), True), StructField('col_a_8', ArrayType(IntegerType(), True), True), StructField('col_a_9', IntegerType(), True), StructField('col_a_10', LongType(), True), StructField('col_a_11', LongType(), True), StructField('col_a_12', LongType(), True), StructField('col_a_13', LongType(), True)])
Schema for table 'table_b': StructType([StructField('col_b_0', StringType(), True), StructField('col_b_1', IntegerType(), True), StructField('col_b_2', StringType(), True), StructField('col_b_3', IntegerType(), True), StructField('col_b_4', LongType(), True), StructField('col_b_5', StringType(), True), StructField(

In [0]:
# -------------------------------------
# Step 5. Process join information.
# -------------------------------------
# Assumption: The final sheet (last sheet) is named "Joins" and holds the join definitions.
join_info_df = sheets[sheet_names[1]]
joins = []
# Here we assume join_info_df has columns: "LeftTable", "LeftColumn", "RightTable", "RightColumn", and optionally "JoinType"
for idx, row in join_info_df.iterrows():
    join_detail = {
        "left_table": row["table1"],
        "right_table": row["table2"],
        "join_method": row["join_method"],
        "left_column": row["column1"],
        "right_column": row["column2"]
    }
    joins.append(join_detail)

print("Join definitions:")
for join in joins:
    print(f"  {join['left_table']}.{join['left_column']} {join['join_method'].upper()} JOIN {join['right_table']}.{join['right_column']}")


Join definitions:
  table_a.col_a_1 LEFT JOIN table_d.col_d_0
  table_a.col_a_1 LEFT JOIN table_e.col_e_0
  table_a.col_a_5 LEFT JOIN table_e.col_e_0
  table_c.col_c_9 LEFT JOIN table_e.col_e_0
  table_c.col_c_10 LEFT JOIN table_e.col_e_0


In [0]:
# -------------------------------------
# (Optional) Step 6. Build a dynamic join query.
# -------------------------------------
# If you later load your data into Spark DataFrames and register them as temporary views,
# you could build and execute a join query dynamically. For example, suppose you have:
#    df_customers = spark.read.csv("customers.csv", schema=schemas["customers"], header=True)
#    df_customers.createOrReplaceTempView("customers")
#    ... and similarly for other tables.
#
# The code below builds a join SQL string assuming sequential joining.
if table_names:
    join_query = f"SELECT * FROM {table_names[0]}"
    for join in joins:
        # Note: This simple logic assumes that the join order is appropriate.
        join_query += (
            f" {join['join_method'].upper()} JOIN {join['right_table']} "
            f"ON {join['left_table']}.{join['left_column']} = {join['right_table']}.{join['right_column']}"
        )
    print("Constructed join query:")
    print(join_query)
    # To execute the query once the tables are registered as temp views:
    # result_df = spark.sql(join_query)
    # result_df.show()

# -------------------------------------
# Now you have:
#  - 'table_names' and 'approx_row_counts' from the overview.
#  - 'table_metadata': a dictionary mapping table names to their metadata DataFrames.
#  - 'schemas': a dictionary mapping table names to Spark schemas.
#  - 'joins': a list of dictionaries describing the join relationships.
#
# You can now use this information to drive your Spark ETL/processing workflow.
#
# When finished, stop the Spark session (if running in a script).
# spark.stop()


Constructed join query:
SELECT * FROM table_a LEFT JOIN table_d ON table_a.col_a_1 = table_d.col_d_0 LEFT JOIN table_e ON table_a.col_a_1 = table_e.col_e_0 LEFT JOIN table_e ON table_a.col_a_5 = table_e.col_e_0 LEFT JOIN table_e ON table_c.col_c_9 = table_e.col_e_0 LEFT JOIN table_e ON table_c.col_c_10 = table_e.col_e_0


# Actually Generate Data

In [0]:
# ========================================
# PART 2: Generate random data for each table and register as temp views
# ========================================

def generate_random_dataframe(schema, num_rows):
    """
    Given a Spark StructType schema and a number of rows, generate a DataFrame with random data
    using Spark’s distributed operations.
    For numerical types, if metadata has "min" and "max", those bounds are used.
    """
    # Start with a DataFrame with a column "id" (this DataFrame is generated in a distributed fashion)
    df = spark.range(num_rows)
    
    # For each field in the schema, add a column with a random value.
    for field in schema.fields:
        col_name = field.name
        dt = field.dataType
        md = field.metadata or {}
        
        if isinstance(dt, (IntegerType, LongType)):
            # Use provided min and max if available; otherwise default to 1 and 1000.
            min_val = md.get("min", 1)
            max_val = md.get("max", 1000)
            expr = (F.rand() * (float(max_val) - float(min_val)) + float(min_val))
            # Cast appropriately.
            if isinstance(dt, IntegerType):
                df = df.withColumn(col_name, expr.cast("int"))
            else:
                df = df.withColumn(col_name, expr.cast("long"))
                
        elif isinstance(dt, (FloatType, DoubleType)):
            min_val = md.get("min", 0.0)
            max_val = md.get("max", 1000.0)
            expr = (F.rand() * (float(max_val) - float(min_val)) + float(min_val))
            if isinstance(dt, FloatType):
                df = df.withColumn(col_name, expr.cast("float"))
            else:
                df = df.withColumn(col_name, expr.cast("double"))
                
        elif isinstance(dt, BooleanType):
            # Generate a boolean value based on a threshold.
            df = df.withColumn(col_name, F.rand() > 0.5)
            
        elif isinstance(dt, DateType):
            # Generate a random date by adding a random number of days (e.g., 0 to 9000) to a base date.
            df = df.withColumn(col_name, F.expr("date_add('2000-01-01', cast(rand() * 9000 as int))"))
            
        elif isinstance(dt, TimestampType):
            # Generate a random timestamp by first generating a random date and then converting it.
            df = df.withColumn(col_name, F.expr("to_timestamp(date_add('2000-01-01', cast(rand() * 9000 as int)))"))
            
        elif isinstance(dt, StringType):
            # Use the built-in uuid() function for random strings.
            df = df.withColumn(col_name, F.expr("uuid()"))
            
        else:
            # For any unrecognized type, set the column to null.
            df = df.withColumn(col_name, F.lit(None))
            
    # Drop the original "id" column.
    return df.drop("id")

# Create and register a DataFrame for each table using the distributed random data generation.
# NOTE: THIS WAS SCALED DOWN FOR TESTING PURPOSES. UNCOMMENT LINE 74 AND COMMENT OUT LINES 68-73 FOR REAL TESTING
dfs = {}
for tbl, count in zip(table_names, approx_row_counts):
    schema = schemas[tbl]
    if tbl == 'table_a':
        num_rows = 100000000
    elif tbl == 'table_c':
        num_rows = 21000000
    else:
        num_rows = int(count)
    # num_rows = int(count)
    df = generate_random_dataframe(schema, num_rows)
    dfs[tbl] = df
    print(f"Created DataFrame for table '{tbl}' with {num_rows} random rows.")

Created DataFrame for table 'table_a' with 100000000 random rows.
Created DataFrame for table 'table_b' with 14000 random rows.
Created DataFrame for table 'table_c' with 21000000 random rows.
Created DataFrame for table 'table_d' with 12000000 random rows.
Created DataFrame for table 'table_e' with 33000000 random rows.


# GroupBys

In [0]:
table_a = dfs['table_a']
gb_test1 = table_a.groupBy(['col_a_1', 'col_a_3', 'col_a_5', 'col_a_7', 'col_a_9']).count()

gb_test1.write.format("noop").mode("overwrite").save()

In [0]:
table_a = dfs['table_a']
gb_test2 = table_a.groupBy(['col_a_1']).count()

gb_test2.write.format("noop").mode("overwrite").save()

# Joins

### table_c `col_1` and `col_3` are both ArrayType(IntegerType) columns in real life, most of the time the array is one value long, but sometimes it's multiple values. Try and figure this out at home.

In [0]:
table_a = dfs['table_a']
table_b = dfs['table_b']

join_test1 = table_a.join(table_b, [
    table_a["col_a_1"]==table_b["col_b_8"],
    table_a["col_a_3"]==table_b["col_b_3"],
    table_a["col_a_5"]==table_b["col_b_9"],
    table_a["col_a_7"]==table_b["col_b_1"],
],
how='left')
join_test1.write.format("noop").mode("overwrite").save()

In [0]:
table_a = dfs['table_a']
table_c = dfs['table_c']

join_test2 = table_a.join(table_c, [
    table_a["col_a_1"]==table_c["col_c_10"],
    table_a["col_a_3"]==table_c["col_c_9"],
    table_a["col_a_9"]==table_c["col_c_11"],
],
how='left')

join_test2.write.format("noop").mode("overwrite").save()

In [0]:
table_a = dfs['table_a']
table_d = dfs['table_d']

join_test3 = table_a.join(table_d, [
    table_a["col_a_1"]==table_d["col_d_0"],
    table_a["col_a_5"]==table_d["col_d_1"],
],
how='left')

join_test3.write.format("noop").mode("overwrite").save()

In [0]:
table_a = dfs['table_a']
table_e = dfs['table_e']

join_test4 = table_a.join(table_e, table_a["col_a_1"]==table_e["col_e_0"], how='left')

join_test4.write.format("noop").mode("overwrite").save()

In [0]:
table_c = dfs['table_c']
table_e = dfs['table_e']

join_test5 = table_c.join(table_e, table_c["col_c_5"]==table_e["col_e_0"], how='left')

join_test5.write.format("noop").mode("overwrite").save()

In [0]:
table_c = dfs['table_c']
table_e = dfs['table_e']

join_test6 = table_c.join(table_e, table_c["col_c_10"]==table_e["col_e_0"], how='left')

join_test6.write.format("noop").mode("overwrite").save()

In [0]:
linked_join_test = (
    table_a
    .join(
        table_b,
        [
            table_a["col_a_1"] == table_b["col_b_8"],
            table_a["col_a_3"] == table_b["col_b_3"],
            table_a["col_a_5"] == table_b["col_b_9"],
            table_a["col_a_7"] == table_b["col_b_1"],
        ],
        how="left"
    )
    .join(
        table_c,
        [
            table_a["col_a_1"] == table_c["col_c_10"],
            table_a["col_a_3"] == table_c["col_c_9"],
            table_a["col_a_9"] == table_c["col_c_11"],
        ],
        how="left"
    )
    .join(
        table_d,
        [
            table_a["col_a_1"] == table_d["col_d_0"],
            table_a["col_a_5"] == table_d["col_d_1"],
        ],
        how="left"
    )
    .join(
        table_e,
        table_a["col_a_1"] == table_e["col_e_0"],
        how="left"
    )
)

linked_join_test.write.format("noop").mode("overwrite").save()